In [1]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
import pandas as pd
import numpy as np
import glob,os
from tqdm.auto import tqdm
import math
import sklearn
from sklearn.linear_model import LinearRegression
import pickle
import rfreg_funcs
from sklearn.metrics import r2_score

In [29]:
precip_df = rfreg_funcs.depickle('./store/rf_feature/8_24_rfreg_v10_firstbatch.pkl')

In [20]:
import sys
sys.path.insert(1,'../')
import preprocess
p1="../../timeseries_csv/ts_wp/"
p2="../../../targets/"

ds47=preprocess._process_dataset(glob.glob(p1+'*2010OMAIS*')[0])
tcwp47=ds47.values
var_names=ds47.columns.values.tolist()

varindices, lagindices = np.linspace(0,233,233+1),np.linspace(-24,-8,17)
var_and_lagsALL = []
for ind,obj in enumerate(varindices):
    var_and_lagsALL.append([(int(obj),int(bb)) for bb in lagindices])

In [7]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
X,y = rfreg_funcs.depickle('./store/Xsaved_8_24'),rfreg_funcs.depickle('./store/ysaved_8_24')

In [30]:
target = 'v10'
r2_train = ([r2_score(y[target]['train'],model.predict(X[target]['train'])) for model in precip_df['models']])
r2_valid = ([r2_score(y[target]['valid'],model.predict(X[target]['valid'])) for model in precip_df['models']])
r2_test_old = ([r2_score(y[target]['test'],model.predict(X[target]['test'])) for model in precip_df['models']])
r2_test_new = ([r2_score(y[target]['newtest'],model.predict(X[target]['newtest'])) for model in precip_df['models']])

In [8]:
v10=np.asarray(r2_test_new).max(),np.asarray(r2_train).max(),np.asarray(r2_test_old).max(),np.asarray(r2_test_old).argmax()
v10

(0.7865097142167998, 0.8789279982491565, 0.7409038798278823, 10)

In [31]:
precip_df['models'][np.asarray(r2_test_old).argmax()]

RandomForestRegressor(max_depth=60, min_samples_leaf=60, min_samples_split=120)

In [23]:
model = precip_df['models'][np.asarray(r2_test_old).argmax()]
target = 'v10'
print([mean_squared_error(y[target]['train'],model.predict(X[target]['train'])),\
  mean_squared_error(y[target]['valid']+y[target]['test'],model.predict(X[target]['valid']+X[target]['test'])),\
  mean_squared_error(y[target]['newtest'],model.predict(X[target]['newtest']))])

[46252.724650715165, 100034.85232630881, 102161.00271408762]


In [32]:
model = precip_df['models'][np.asarray(r2_test_old).argmax()]
target = 'v10'
print([mean_absolute_error(y[target]['train'],model.predict(X[target]['train'])),\
  mean_absolute_error(y[target]['valid']+y[target]['test'],model.predict(X[target]['valid']+X[target]['test'])),\
  mean_absolute_error(y[target]['newtest'],model.predict(X[target]['newtest']))])

[1.5040293409175347, 2.174481973996568, 1.9955470466586385]


In [24]:
precip=np.asarray(r2_test_new).max(),np.asarray(r2_train).max(),np.asarray(r2_test_old).max()
precip

(0.5819247942533833, 0.7517649272314094, 0.6500373708737699)

In [20]:
pmin=np.asarray(r2_test_new).max(),np.asarray(r2_train).max(),np.asarray(r2_test_old).max()
pmin

(0.8884124116911621, 0.9253897631544334, 0.7698719550029873)

In [55]:
#permute_precip_bestmodel = rfreg_funcs._get_importance_ranking(model=precip_df['models'][8],TYPE='permutation',index=var_and_lagsALL,X=X['precip'],y=y['precip'])
gini_precip_bestmodel = rfreg_funcs._get_importance_ranking(model=precip_df['models'][int(np.asarray(r2_test_new).argmax())],\
                                                          TYPE='gini',index=var_and_lagsALL,X=X['v10'],y=y['v10'])
gini_importance_store = rfreg_funcs.sorted_importance(importances=gini_precip_bestmodel)

In [56]:
import importlib
importlib.reload(rfreg_funcs)
rfreg_funcs.save_to_pickle('./store/rf_feature/rfbest_v10_gini_8_24.pkl',list(gini_importance_store['Combination']))

In [28]:
gini_importance_store['Combination']

0          (1, -8)
1        (15, -24)
2         (42, -8)
3         (43, -8)
4       (191, -13)
           ...    
3973     (92, -17)
3974     (92, -16)
3975     (92, -15)
3976     (92, -13)
3977     (233, -8)
Name: Combination, Length: 3978, dtype: object

In [1]:
TABLEname = 'CI_Paper_Results'
path_TXT = './'
path = path_TXT+TABLEname+".txt"

topline1 = " & & Training Set & \\textbf{Validation Set} & Training Set & Training Set\\tabularnewline\n"
topline2 = "Trained & Model & Same temperature & Same temperature & \\textbf{Different temperature} & Same temperature\\tabularnewline\n"
topline3 = "on & & Same geography & Same geography & Same geography & \\textbf{Different geography}\\tabularnewline\n"

caption = "Mean-Squared Error (MSE, in units W$^{2}$ m$^{-4}$) of six models trained in three simulations (first column)"+\
" and evaluated over the training or validation set of the same and two other simulations (last four columns)."+\
" The models (second column) are brute-force (BF) or climate-invariant (CI), multiple linear regressions (MLR) "+\
"or neural nets (NN), and sometimes include dropout layers preceded by a batch normalization layer (DN)."+\
" The models are trained for 20 epochs and we give the MSE corresponding to the epoch of minimal validation loss"+\
" followed by the MSE averaged over the 5 epochs with lowest validation losses (in parentheses)."

COLOR = ['black','purple','black','teal','purple','teal']
MODEL = [[" & MLR BF",
          "\\textcolor{blue}{Cold} & \\textcolor{purple}{MLR CI}",
          "\\textcolor{blue}{Aquaplanet} & NN BF",
          "\\textcolor{blue}{(-4K)} & \\textcolor{teal}{NN BF+DN}",
          "SPCAM3 & \\textcolor{purple}{NN CI}",
          " & \\textcolor{teal}{NN CI+DN}"],
         [" & MLR BF",
          "\\textcolor{red}{Warm} & \\textcolor{purple}{MLR CI}",
          "\\textcolor{red}{Aquaplanet} & NN BF",
          "\\textcolor{red}{(+4K)} & \\textcolor{teal}{NN BF+DN}",
          "SPCAM3 & \\textcolor{purple}{NN CI}",
          " & \\textcolor{teal}{NN CI+DN}"],
         [" & MLR BF",
          "\\textcolor{green}{Cold} & \\textcolor{purple}{MLR CI}",
          "\\textcolor{green}{Earth-like} & NN BF",
          "\\textcolor{green}{(-4K)} & \\textcolor{teal}{NN BF+DN}",
          "SPCAM5 & \\textcolor{purple}{NN CI}",
          " & \\textcolor{teal}{NN CI+DN}"]]

In [ ]:
MSE_array = [0,0,0

In [3]:
f= open(path,"w+")
f.write("\\begin{table*}\n")
f.write("\\begin{centering}\n")
f.write("\\begin{tabular}{c|c||c|c|c|c}\n")
f.write(topline1)
f.write(topline2)
f.write(topline3)
for itrain in range(3):
    f.write("\\hline\n")
    for imodel in range(6):
        f.write(MODEL[itrain][imodel])
        for itest in range(4):
            f.write(" & ")
            f.write("\\textcolor{"+COLOR[imodel]+"}{")
            f.write("%3.0f"%0.677+" ")
            f.write("("+"%3.0f"%0.988+") ")
            f.write("}")
        f.write("\\tabularnewline\n")
f.write("\\end{tabular}\n")
f.write("\\par\\end{centering}\n")
f.write("\n")
f.write("\\caption{")
f.write(caption)
f.write("}\n")
f.write("\\label{tab:Results}\n")
f.write("\\end{table*}\n")
f.close()
print('Table',TABLEname,'printed to',path)

Table CI_Paper_Results printed to ./CI_Paper_Results.txt


In [ ]:

\begin{table}
\begin{centering}
\begin{tabular}{cc|c|c|c|c|c|c|c|c|c|c}
\multicolumn{2}{c}{ML Models} & \multicolumn{3}{c}{Training} & \multicolumn{3}{c}{Validation} & \multicolumn{3}{c|}{Test } & \tabularnewline
\hline 
\multicolumn{2}{c|}{\textbf{Target}} & Pmin (hPa)\textcolor{purple}{{} } & V10 (m/s) & Precip (km\textasciicircum 3) & Pmin (hPa)\textcolor{purple}{{} } & V10 (m/s) & Precip (km\textasciicircum 3) & Pmin (hPa)\textcolor{purple}{{} } & V10 (m/s) & Precip (km\textasciicircum 3) & \tabularnewline
\cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} 
\multicolumn{2}{c|}{\textbf{Causal RF}} & 5.3 & 11 & 11 & 11 & 11 & 11 & 11 & 11 & 11 & \tabularnewline
\hline 
\multicolumn{2}{c|}{\textbf{Causal MLR}} &  &  &  &  &  &  &  &  &  & \tabularnewline
\hline 
\multirow{3}{*}{\textbf{Non-causal RF}} & All & 0.93 & 0.88 & 0.75 & 0.77 & 0.74 & 0.65 & 0.89 & 0.79 & 0.58 & \tabularnewline
\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} 
 & Lagged &  &  &  &  &  &  &  &  &  & \tabularnewline
\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} 
 & Random &  &  &  &  &  &  &  &  &  & \tabularnewline
\hline 
\multirow{4}{*}{\textbf{Non-causal MLR}} & All & 0.99 & 0.98 & 0.96 & -0.94 & -10.85 & -127.98 & 0.51 & -0.01 & -0.39 & \tabularnewline
\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} 
 & Lagged &  &  &  &  &  &  &  &  &  & \tabularnewline
\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} 
 & Random &  &  &  &  &  &  &  &  &  & \tabularnewline
\cline{2-12} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} 
 & XAI &  &  &  &  &  &  &  &  &  & \tabularnewline
\cline{1-1} \cline{3-12} \cline{4-12} \cline{5-12} \cline{6-12} \cline{7-12} \cline{8-12} \cline{9-12} \cline{10-12} \cline{11-12} \cline{12-12} 
\multicolumn{2}{c|}{\textbf{LSTM}} & 0.87 & 0.81 & 0.71 & 0.77 & 0.75 & 0.65 & 0.81 & 0.75 & 0.61 & \tabularnewline
\hline 
\end{tabular}
\par\end{centering}
\caption{}

\end{table}